#### Pythonとは
* シンプルで覚えることが少ない構文
* 1行で多くの処理を記述可能
* 標準ライブラリとして、言語に多くの機能があらかじめ用意されている
* さらに幅広い用途に使えるPython向け外部ライブラリが豊富に存在している

という特徴があり、プログラミングを始めるに持ってこいの言語。


###### 出力
出力にはprint(出力したいもの）を用いる

In [ ]:
##出力
print("Hello!!")

###### 変数
a = [データ]　とすると、aという箱にデータを格納した　というイメージとなる。これを”変数に値を代入する"という。

In [ ]:
a = 192
print(a)

###### 四則演算
以下のように四則演算が可能

In [ ]:
print(3+4)
print(3-4)
print(3*4)
print(3**4)
print(13/4)
print(13//4)
print(13%4)
print("aaa"+"bbb")

###### 計算規則
()の中から計算、他は左から順に計算

In [ ]:
print(8-3+4+2*0.6)
print(3-3**3+49//7)

演習
適宜、**調べる**　が大事です。

In [ ]:
import sys
import gc
import os
import random
import time
from contextlib import contextmanager
from pathlib import Path
from collections import defaultdict, Counter

import cv2
from PIL import Image
import numpy as np
import pandas as pd
import scipy as sp
import pydicom
import sklearn.metrics
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss
from functools import partial
from tqdm import tqdm
from sklearn.metrics import precision_score,recall_score,f1_score,log_loss
from  sklearn.metrics import accuracy_score as acc
import torch
import torch.nn as nn
from torch.optim import Adam, SGD
from torch.optim.lr_scheduler import CosineAnnealingLR, ReduceLROnPlateau
from torch.utils.data import DataLoader, Dataset
import torchvision.models as models
from sklearn.model_selection import train_test_split
#from aug import RandomAugMix
from albumentations import Compose, Normalize, HorizontalFlip, VerticalFlip,RandomGamma, RandomRotate90,GaussNoise,Cutout
from albumentations.pytorch import ToTensorV2
!pip install geffnet
import geffnet
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
def seed_torch(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed_torch(seed=42)
SIZE = 512

In [ ]:
import glob
j = glob.glob("../input/agriculture-crop-images/crop_images/jute/*")
r = glob.glob("../input/agriculture-crop-images/crop_images/rice/*")
df_j = pd.DataFrame()
df_j["path"] = j
df_j["label"] = 0
df_r = pd.DataFrame()
df_r["path"] = r
df_r["label"] = 1
df = pd.concat([df_r,df_j],axis =0)
df.head()

In [ ]:
def func0(x):
    return os.path.isfile(x)
df["exist"] = df["path"].map(func0)

In [ ]:
##train,testに分割,trainをtrain,validにまた分割
train,test = train_test_split(df)
train = train.reset_index(drop=True)
folds = train.copy()
kf = StratifiedKFold(n_splits=4, shuffle=True, random_state=2020)
for fold, (train_index, val_index) in enumerate(kf.split(folds.values, folds["label"])):
    print("num_train,val",len(train_index),len(val_index),len(val_index)+len(train_index))
    folds.loc[val_index, 'fold'] = int(fold)

In [ ]:
# Utils
# ====================================================

@contextmanager
def timer(name):
    t0 = time.time()
    LOGGER.info(f'[{name}] start')
    yield
    LOGGER.info(f'[{name}] done in {time.time() - t0:.0f} s.')

    
def init_logger(log_file='train.log'):
    from logging import getLogger, DEBUG, FileHandler,  Formatter,  StreamHandler
    
    log_format = '%(asctime)s %(levelname)s %(message)s'
    
    stream_handler = StreamHandler()
    stream_handler.setLevel(DEBUG)
    stream_handler.setFormatter(Formatter(log_format))
    
    file_handler = FileHandler(log_file)
    file_handler.setFormatter(Formatter(log_format))
    
    logger = getLogger('RSNA2020')
    logger.setLevel(DEBUG)
    logger.addHandler(stream_handler)
    logger.addHandler(file_handler)
    
    return logger
import datetime
dt_now = datetime.datetime.now()
print("実験開始",dt_now)
LOG_FILE = 'train{}.log'.format(dt_now)
LOGGER = init_logger(LOG_FILE)

In [ ]:
class TrainDataset(Dataset):
    def __init__(self, df,transform1=None, transform2=None):
        self.df = df
        self.transform = transform1
        self.transform_ = transform2
        
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        file_path = self.df['path'].values[idx]
        image = cv2.imread(file_path)
        try:
            image = cv2.resize(image,(SIZE,SIZE))
        except Exception as e:
            print(file_path)
        label_ = self.df["label"].values[idx]
        if self.transform:
            image = self.transform(image=image)['image']
        if self.transform_:
            image = self.transform_(image=image)['image']
        #label = torch.eye(5)[torch.tensor(label_)]
        label = torch.tensor(label_)
        
        return image, label

In [ ]:
class efenet_Model(nn.Module):

    def __init__(self):
        super().__init__()
        self.model = geffnet.efficientnet_b0(pretrained=True, drop_rate=0.25)
        self.model.classifier = nn.Linear(self.model.classifier.in_features, 1)
    def forward(self, x):
        x = self.model(x)
        return x
class inf_efnet_model(nn.Module):

    def __init__(self):
        super().__init__()
        self.model = geffnet.efficientnet_b0(pretrained=False, drop_rate=0.25)
        self.model.classifier = nn.Linear(self.model.classifier.in_features, 1)  
    def forward(self, x):
        x = self.model(x)
        return x

In [ ]:
def get_transforms1(*, data):

    #train,valid以外だったら怒る
    
    if data == 'train':
        return Compose([
            HorizontalFlip(p=0.5),
            VerticalFlip(p=0.5),
            #GaussNoise(p=0.5),
            #RandomRotate90(p=0.5),
            #RandomGamma(p=0.5),
            Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])])
    
    elif data == 'valid':
        return Compose([
            Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
            )
        ])

def to_tensor(*args):

        return Compose([
            ToTensorV2()
        ])

In [ ]:
class CFG:
    debug=False
    lr=1e-4
    batch_size=16
    epochs=10# you can train more epochs
    seed=777
    target_size=1
    n_fold=4
    warmup=-1
    device=1
    under_sample=True

In [ ]:
from  sklearn.metrics import roc_auc_score 
def AUC(y_true,y_pred):
    return roc_auc_score(y_true,y_pred)

In [ ]:
def train_fn(fold):
    print(f"### fold: {fold} ###")

        
    trn_idx = folds[folds['fold'] != fold].index
    val_idx = folds[folds['fold'] == fold].index
    train_dataset = TrainDataset(folds.loc[trn_idx].reset_index(drop=True), 
                                 transform1=get_transforms1(data='train'),transform2=to_tensor())#
    valid_dataset = TrainDataset(folds.loc[val_idx].reset_index(drop=True), 
                                 transform1=get_transforms1(data='valid'),transform2=to_tensor())#

    train_loader = DataLoader(train_dataset, batch_size=CFG.batch_size, shuffle=True, num_workers=4)
    valid_loader = DataLoader(valid_dataset, batch_size=CFG.batch_size, shuffle=False, num_workers=4)
    model = efenet_Model()
    
    
    model.to(device)
    
    optimizer = Adam(model.parameters(), lr=CFG.lr, amsgrad=False)
    
    criterion = nn.BCELoss()
    best_score = -100
    best_loss = np.inf
    best_preds = None
    
    
    for epoch in range(CFG.epochs):
        
        start_time = time.time()

        model.train()

        if CFG.warmup>epoch:
            print("freeze")
            for param in model.model.parameters():
                param.requires_grad = False
            for param in model.model.classifier.parameters():
                param.requires_grad = True
        
        if CFG.warmup<=epoch:
            print("unfreeze")
            for param in model.parameters():
                param.requires_grad = True
        avg_loss = 0.

        optimizer.zero_grad()
        tk0 = tqdm(enumerate(train_loader), total=len(train_loader))

        for i, (images, labels) in tk0:

            images = images.to(device)
            labels = labels.to(device)
            
            y_preds = model(images.float())
            y_preds = torch.sigmoid(y_preds.view(-1))
            """
            if i ==0:
                print(y_preds.dtype,labels.dtype)
                print(y_preds.shape,labels.shape)"""
            loss = criterion(y_preds, labels.float())
            
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            avg_loss += loss.item() / len(train_loader)
        model.eval()
        avg_val_loss = 0.
        preds = []
        valid_labels = []
        tk1 = tqdm(enumerate(valid_loader), total=len(valid_loader))

        for i, (images, labels) in tk1:
            
            images = images.to(device)
            labels = labels.to(device)
            
            with torch.no_grad():
                y_preds = model(images.float())
                
                y_preds = torch.sigmoid(y_preds.view(-1))
            preds.append(y_preds.to('cpu').numpy())
            valid_labels.append(labels.to('cpu').numpy())

            loss = criterion(y_preds,labels.float())
            avg_val_loss += loss.item() / len(valid_loader)
            
        preds = np.concatenate(preds)
        valid_labels = np.concatenate(valid_labels)

        print(preds.shape,valid_labels.shape)

        score = AUC(valid_labels,preds)

        elapsed = time.time() - start_time
        LOGGER.debug(f'  Epoch {epoch+1} - avg_train_loss: {avg_loss:.6f}  avg_val_loss: {avg_val_loss:.6f}  time: {elapsed:.0f}s')
        
        if score>best_score:#loglossのスコアが良かったら予測値を更新...best_epochをきめるため
            best_score = score
            best_preds = preds
            LOGGER.debug(f'  Epoch {epoch+1} - Save Best Score: {best_score:.4f}')
            torch.save(model.state_dict(), f'fold{fold}_baseline.pth')#各epochのモデルを保存。。。best_epoch終了時のモデルを推論に使用する？
    

    return best_preds, valid_labels

In [ ]:
"""
preds = []
valid_labels = []
for fold in range(CFG.n_fold):
    _preds, _valid_labels = train_fn(fold)
    preds.append(_preds)
    valid_labels.append(_valid_labels)
preds = np.concatenate(preds)
valid_labels = np.concatenate(valid_labels)"""

In [ ]:
"""

score = AUC(valid_labels,preds)
import datetime
from sklearn import metrics
dt_now = datetime.datetime.now()
print("現在時刻",dt_now)
print("=====AUC(CV)======",score)

acc = metrics.accuracy_score(valid_labels,preds>0.5)
print('  =====acc(CV)====== {}'.format(acc))"""